# 4.0 LangGraph Quick Introduction

This notebook provides an introduction to **LangGraph 1.0+**, the framework for building stateful, multi-actor applications with LLMs.

**What you'll learn:**
- Core LangGraph concepts: State, Nodes, and Edges
- Building a simple agent with tools
- Memory and persistence with checkpointers
- Building chatbots with conversation history

Check out the [LangGraph documentation](https://langchain-ai.github.io/langgraph/concepts/#background-agents-ai-workflows-as-graphs) for more details.

## Setup

Install the required packages:

In [ ]:
# LangChain 1.0+ and LangGraph 1.0+ Setup
%pip install -qU langchain>=1.0.0 langgraph>=1.0.0
%pip install -qU langchain-openai
%pip install -qU langchain-tavily  # New package for Tavily search tools
%pip install -qU tiktoken

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGCHAIN_API_KEY")

# Enable LangSmith tracing (optional but recommended)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langgraph-introduction"

## LangGraph Core Concepts

LangGraph models agent workflows as **state machines**. You define behavior using three key components:

| Component | Description |
|-----------|-------------|
| **State** | A shared data structure representing the current snapshot of your application. Typically a `TypedDict` or Pydantic `BaseModel`. |
| **Nodes** | Python functions that receive the current State, perform computation or side-effects, and return an updated State. |
| **Edges** | Control flow rules determining which Node to execute next based on the current State. Can be conditional or fixed. |

By composing Nodes and Edges, you can create complex, looping workflows that evolve the State over time.

## Part 1: Building a Simple Agent with LangGraph

Let's build an agent that can use tools. This demonstrates:
- Defining custom tools with the `@tool` decorator
- Creating a state graph with conditional routing
- Using the `ToolNode` prebuilt component

In [ ]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [ ]:
# Define tools for the agent to use
@tool
def search(query: str) -> str:
    """Search the web for current information."""
    # Placeholder implementation - in production, use TavilySearch
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools = [search]
tool_node = ToolNode(tools)

# Bind tools to the model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [ ]:
# Define routing logic: continue to tools or end?
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Determine whether to use tools or finish."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the LLM makes a tool call, route to the tools node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we're done
    return END

In [ ]:
# Define the node that calls the model
def call_model(state: MessagesState):
    """Invoke the model with the current messages."""
    messages = state["messages"]
    response = model.invoke(messages)
    # Return as list to append to existing messages
    return {"messages": [response]}

In [ ]:
# Build the graph
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entry point
workflow.set_entry_point("agent")

# Add conditional edge from agent
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# After tools, always go back to agent
workflow.add_edge("tools", "agent")

# Add memory for persistence
checkpointer = MemorySaver()

# Compile the graph
app = workflow.compile(checkpointer=checkpointer)

In [ ]:
# Visualize the graph
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# Run the agent
result = app.invoke(
    {"messages": [HumanMessage(content="What is the weather in San Francisco?")]},
    config={"configurable": {"thread_id": "1"}}
)

print(result["messages"][-1].content)

In [ ]:
# View the full message history
for msg in result["messages"]:
    print(f"{msg.__class__.__name__}: {msg.content[:100] if msg.content else '[tool call]'}...")

## Part 2: Building a Simple Chatbot

Now let's build a simpler chatbot without tools - just a model with memory.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# The model can maintain context when given message history
response = llm.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
])

print(response.content)

### Creating a Chatbot with LangGraph Persistence

For a true chatbot experience, we use LangGraph's built-in persistence to maintain conversation history across turns.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define the graph
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    """Call the model with current messages."""
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Add single node and edge
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
chatbot = workflow.compile(checkpointer=memory)

In [ ]:
# Visualize
display(Image(chatbot.get_graph().draw_mermaid_png()))

In [ ]:
# Chat with memory - same thread_id maintains conversation
config = {"configurable": {"thread_id": "chat-123"}}

# First message
output = chatbot.invoke({"messages": [HumanMessage("Hi! I'm Bob.")]}, config)
output["messages"][-1].pretty_print()

In [ ]:
# Follow-up - the chatbot remembers!
output = chatbot.invoke({"messages": "What is my name?"}, config)
output["messages"][-1].pretty_print()

In [ ]:
# View full conversation history
output

## Part 3: Chatbot with Custom System Prompt

Let's add a system prompt to customize the chatbot's personality.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a prompt with a custom system message
prompt = ChatPromptTemplate.from_messages([
    ("system", "You talk like a pirate. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
pirate_chatbot = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "pirate-chat-1"}}

output = pirate_chatbot.invoke(
    {"messages": [HumanMessage("Hi! I'm Jim.")]}, 
    config
)
output["messages"][-1].pretty_print()

In [ ]:
# Test memory
output = pirate_chatbot.invoke(
    {"messages": [HumanMessage("What is my name?")]}, 
    config
)
output["messages"][-1].pretty_print()

## Part 4: Adding Custom State Variables

Sometimes you need to track more than just messages. Let's create a chatbot that responds in a user-specified language.

In [ ]:
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph import END
from langgraph.graph.message import add_messages


# Custom state with language parameter
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


# Prompt that uses the language variable
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages"),
])

In [ ]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
workflow.add_edge("model", END)

memory = MemorySaver()
multilingual_chatbot = workflow.compile(checkpointer=memory)

In [ ]:
display(Image(multilingual_chatbot.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "french-chat"}}

output = multilingual_chatbot.invoke(
    {
        "messages": [HumanMessage("Hi! I'm Bob.")],
        "language": "French"
    },
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
# Continue the conversation
output = multilingual_chatbot.invoke(
    {"messages": [HumanMessage("What is my name?")]},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
# View full state
output

## Part 5: Using the New `create_agent` API (LangChain 1.0+)

LangChain 1.0 introduces `create_agent` - a simplified way to create agents without manually building graphs.

**Key features:**
- Single function call to create an agent
- Model can be specified as a string (`"openai:gpt-4o-mini"`)
- Built-in support for tools, system prompts, and middleware
- Returns a `CompiledStateGraph` that you can invoke directly

In [ ]:
from langchain.agents import create_agent
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # Placeholder - in production use a real weather API
    return f"The weather in {location} is 72°F and sunny."


# Create agent with the new API
agent = create_agent(
    model="openai:gpt-4o-mini",  # String format!
    tools=[get_weather],
    system_prompt="You are a helpful weather assistant. Always be concise.",
)

In [ ]:
# Invoke the agent - note the messages format
result = agent.invoke({
    "messages": [{"role": "user", "content": "What's the weather in Tokyo?"}]
})

print(result["messages"][-1].content)

In [ ]:
# Stream the response
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What's the weather in Paris?"}]},
    stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

### Using TavilySearch with create_agent

Let's create an agent that can search the web using the new `langchain-tavily` package.

In [ ]:
from langchain_tavily import TavilySearch

# Create the Tavily search tool
tavily_search = TavilySearch(
    max_results=3,
    topic="general",
)

# Create agent with search capability
search_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[tavily_search],
    system_prompt="You are a helpful research assistant. Use the search tool to find current information. Always cite your sources.",
)

In [ ]:
# Search for current information
result = search_agent.invoke({
    "messages": [{"role": "user", "content": "What are the latest developments in LangChain 1.0?"}]
})

print(result["messages"][-1].content)

## Summary

In this notebook, you learned:

1. **LangGraph Fundamentals**: State, Nodes, and Edges for building stateful AI workflows
2. **Building Agents Manually**: Creating agents with tools using `StateGraph`, `ToolNode`, and conditional edges
3. **Persistence**: Using `MemorySaver` to maintain conversation history across turns
4. **Custom State**: Extending `MessagesState` with additional variables like `language`
5. **LangChain 1.0 `create_agent`**: The new simplified API for creating agents

### Key LangChain 1.0 Changes

| Old Pattern | New Pattern |
|------------|-------------|
| `AgentExecutor` | `create_agent()` returns a graph |
| `create_react_agent()` | `create_agent()` |
| `TavilySearchResults` | `TavilySearch` from `langchain-tavily` |
| Instantiate model classes | String format: `"openai:gpt-4o-mini"` |